## Load Model

In [4]:
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.callbacks import ModelCheckpoint

from modules.dataset import Dataset
from modules.models import Model
from modules.parser import parse_option

# Constant variables
NAME = "Resnet50"
EPOCHS = 10
NUM_FOLDS = 5

# Model Checkpoint
dataset = Dataset()

# Modeling
transfer_model = tf.keras.applications.resnet50.ResNet50(
    include_top=False, 
    weights="imagenet",
    input_shape=(224, 224, 3),
    pooling=None
)
model_wandb = Model(transfer_model).get_model(flatten=True)
model_wandb.summary()

best_model = wandb.restore('model-best.h5', run_path="chestxray/Experiment 3/30zscnm2")
model_wandb.load_weights(best_model.name)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 128)               12845184  
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 15)                9

In [6]:
model_checkpoint = tf.keras.models.load_model("results/models/best_model.h5")
model_checkpoint.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 128)               12845184  
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 15)                9

## Preprocessing

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
from modules.dataset import Dataset

In [8]:
def get_y_true(data):
    y_true=[]
    for X,y in data:
        for label in y:
            y_true.append(label)
    y_true = tf.Variable(y_true)
    return y_true

In [9]:
# Constant variables
# NAME = "ResNet50"
EPOCHS = 10
NUM_FOLDS = 5

dataset = Dataset()
fold_num = 1

# Dataset
train_dataset, test_dataset = dataset.get_kfold(fold_num, sample=True)

In [15]:
y_true = get_y_true(test_dataset)

## Wandb

In [11]:
y_preds = model_wandb.predict(test_dataset)

2022-09-28 08:18:36.267441: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-09-28 08:18:37.530147: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-28 08:18:37.531370: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-28 08:18:37.531473: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-09-28 08:18:37.533115: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-28 08:18:37.533247: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


      6/Unknown - 7s 25ms/step

2022-09-28 08:18:40.832179: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


13/13 [==============================] - 9s 177ms/step


In [12]:
y_true, y_preds

(<tf.Variable 'Variable:0' shape=(200, 15) dtype=int64, numpy=
 array([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]])>,
 array([[0.46522707, 0.10514026, 0.03586749, ..., 0.09740598, 0.12876353,
         0.03058107],
        [0.4655787 , 0.11624805, 0.04319234, ..., 0.10785766, 0.1415401 ,
         0.03622847],
        [0.46227956, 0.10630222, 0.03661184, ..., 0.09855667, 0.13001259,
         0.03125215],
        ...,
        [0.461406  , 0.10941712, 0.03901778, ..., 0.10285376, 0.1312578 ,
         0.03210022],
        [0.46523467, 0.11905814, 0.04422769, ..., 0.1126479 , 0.14279562,
         0.0375489 ],
        [0.45912176, 0.11083672, 0.03956914, ..., 0.10209166, 0.13383913,
         0.03365023]], dtype=float32))

In [13]:
y_true.shape, y_preds.shape

(TensorShape([200, 15]), (200, 15))

In [14]:
m = tf.keras.metrics.AUC(multi_label=True)
m.update_state(y_true, y_preds)
m.result().numpy()

0.45929083

## Tensorflow Keras Model Checkpoint

In [16]:
y_preds = model_checkpoint.predict(test_dataset)

13/13 [==============================] - 2s 28ms/step


In [17]:
y_true, y_preds

(<tf.Variable 'Variable:0' shape=(200, 15) dtype=int64, numpy=
 array([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]])>,
 array([[0.46522707, 0.10514026, 0.03586749, ..., 0.09740598, 0.12876353,
         0.03058107],
        [0.4655787 , 0.11624805, 0.04319234, ..., 0.10785766, 0.1415401 ,
         0.03622847],
        [0.46227956, 0.10630222, 0.03661184, ..., 0.09855667, 0.13001259,
         0.03125215],
        ...,
        [0.461406  , 0.10941712, 0.03901778, ..., 0.10285376, 0.1312578 ,
         0.03210022],
        [0.46523467, 0.11905814, 0.04422769, ..., 0.1126479 , 0.14279562,
         0.0375489 ],
        [0.45912176, 0.11083672, 0.03956914, ..., 0.10209166, 0.13383913,
         0.03365023]], dtype=float32))

In [18]:
y_true.shape, y_preds.shape

(TensorShape([200, 15]), (200, 15))

In [19]:
m = tf.keras.metrics.AUC(multi_label=True)
m.update_state(y_true, y_preds)
m.result().numpy()

0.45929083